In [ ]:
from transformers_lit import load_pretrained_model, load_pretrained_tokenizer
from transformers.generation.utils import GenerationConfig
import torch

In [3]:
# Atom-7B load ok, infer 太慢
model_name = "Chinese-Llama-2-7b"
model_path = f"../models/{model_name}"

In [8]:
from transformers import AutoTokenizer, AutoModel, TextStreamer
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

In [5]:
from transformers import AutoModelForCausalLM
# model = AutoModel.from_pretrained(model_path).half().cuda()

model = AutoModelForCausalLM.from_pretrained(
        model_path,
        return_dict=True,
        device_map="cuda:0",
        # trust_remote_code=True,
        local_files_only=True,
        # use_safetensors=True
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [13]:
instruction = """<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
            If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\n{user_input} [/INST]"""

prompt = instruction.format(user_input="你的名字是什么？")
input_ids = tokenizer("use C# write HELLO", return_tensors='pt').input_ids.cuda()
generate_ids = model.generate(input_ids, 
                              pad_token_id=tokenizer.pad_token_id,
                              eos_token_id=tokenizer.eos_token_id,
                              max_new_tokens=128, 
                              streamer=streamer)


 你的名字是什么？为什么？名字是父母给的，是父母的爱，是父母的希望，是父母的寄托。我叫李小龙，是父母的爱

KeyboardInterrupt: 

In [ ]:
def ask(user_input: str):
   global model, tokenizer
   messages = []
   messages.append({"role": "user", "content": user_input})
   response = model.chat(tokenizer, messages)
   print(response)